In [1]:
from experiments import UniformMeshes, discretize, continuous_coefficient_3d
import numpy as np
import pandas as pd
import torch
import os
import json
import pyamgx
import tqdm

/usr/local/lib/python3.12/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


AMGX version 2.5.0
Built on Dec  8 2025, 14:48:22
Compiled with CUDA Runtime 12.6, using CUDA driver 12.8
The AMGX_initialize_plugins API call is deprecated and can be safely removed.


In [2]:
mesh_family = UniformMeshes(d=3, m=7)

In [3]:
def test_config(cfg, problem, rep_setup=3, rep_solve=5):
    rsc = pyamgx.Resources().create_simple(cfg)

    A = pyamgx.Matrix().create(rsc)
    b = pyamgx.Vector().create(rsc)
    x = pyamgx.Vector().create(rsc)

    A.upload_CSR(problem.exact_form_matrix)
    b.upload(problem.load_vector)

    solver = pyamgx.Solver().create(rsc, cfg)

    setup_times = []
    for _ in range(rep_setup):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        solver.setup(A)
        end.record()
        torch.cuda.synchronize()
        setup_times.append(start.elapsed_time(end) / 1000)

    solve_times = []
    for _ in range(rep_solve):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        x.set_zero(n=problem.load_vector.shape[0], block_dim=1)
        start.record()
        solver.solve(b, x, zero_initial_guess=True)
        end.record()
        torch.cuda.synchronize()
        solve_times.append(start.elapsed_time(end) / 1000)

    x_vec = np.zeros_like(problem.load_vector)
    x.download(x_vec)
    residual = np.linalg.norm(problem.exact_form_matrix @ x_vec - problem.load_vector)

    A.destroy()
    b.destroy()
    x.destroy()
    solver.destroy()
    rsc.destroy()

    return {
        "setup_time": min(setup_times),
        "solve_time": min(solve_times),
        "residual": residual,
    }

In [4]:
def patch_config(cfg):
    cfg = cfg.copy()
    cfg["solver"]["print_grid_stats"] = 0
    cfg["solver"]["print_solve_stats"] = 0
    cfg["solver"]["obtain_timings"] = 0
    cfg["solver"]["convergence"] = "RELATIVE_INI"
    cfg["solver"]["tolerance"] = 1e-9
    cfg["solver"]["max_iters"] = 1000
    if "preconditioner" in cfg:
        cfg["preconditioner"]["print_grid_stats"] = 0
        cfg["preconditioner"]["print_solve_stats"] = 0
    if "preconditioner" in cfg["solver"]:
        cfg["solver"]["preconditioner"]["print_grid_stats"] = 0
        cfg["solver"]["preconditioner"]["print_solve_stats"] = 0
    return cfg

In [5]:
configs = {}
for filename in tqdm.tqdm(os.listdir("/workspace/AMGX/src/configs/")):
    if filename.endswith(".json"):
        with open(f"/workspace/AMGX/src/configs/{filename}") as f:
            cfg_json = patch_config(json.load(f))
            name = filename[:-5]
            if "GMRES" in cfg_json["solver"]["solver"]:
                cfg_json["solver"]["solver"] = "PCG"
                name += "__MOD_PCG"
            configs[name] = cfg_json

100%|██████████| 63/63 [00:00<00:00, 24973.17it/s]


In [6]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_3d.problem, mesh_family["S3"]
)

In [7]:
results = []
for name, cfg_json in tqdm.tqdm(configs.items()):
    cfg = pyamgx.Config().create_from_dict(cfg_json)
    results.append(
        {
            **test_config(
                cfg, double_precision_discrete_problem, rep_setup=1, rep_solve=1
            ),
            "config": name,
        }
    )
    cfg.destroy()

df = pd.DataFrame(results)

 94%|█████████▎| 58/62 [02:13<00:04,  1.14s/it]!!! detected some memory leaks in the code: trying to free non-empty temporary device pool !!!
ptr:     0x7275f82fd000 size: 4096
100%|██████████| 62/62 [02:15<00:00,  2.19s/it]


In [8]:
df

,setup_time,solve_time,residual,config
0,0.033582,3.449075,NaN,AGGREGATION_DILU
1,0.011584,0.117435,3.924485e-09,AGGREGATION_GS
2,0.072903,0.421364,NaN,AGGREGATION_JACOBI
3,0.006468,0.420620,NaN,AGGREGATION_LOW_DEG_BJ
4,0.019112,3.408465,NaN,AGGREGATION_LOW_DEG_DILU
...,...,...,...,...
57,0.018101,0.118773,1.877222e-09,V-cheby-smoother
58,0.010458,0.078142,2.242131e-09,V-cheby_poly-smoother
59,0.011007,0.432268,NaN,V
60,0.011020,0.784871,NaN,W


In [9]:
working_configs = list(df[df.residual < 1e-6]["config"])
working_configs

['AGGREGATION_GS',
 'AGGREGATION_LOW_DEG_GS',
 'AGGREGATION_THRUST_GS',
 'AMG_AGGRREGATION_CG',
 'AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC',
 'AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG',
 'AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_L1_AGGRESSIVE_HMIS__MOD_PCG',
 'AMG_CLASSICAL_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_PMIS__MOD_PCG',
 'CG_DILU',
 'FGMRES_CLASSICAL_AGGRESSIVE_HMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG',
 'FGMRES_NOPREC__MOD_PCG',
 'GMRES__MOD_PCG',
 'GMRES_AMG_D2__MOD_PCG',
 'PBICGSTAB_NOPREC',
 'PCG_NOPREC',
 'V-cheby-aggres-L1-trunc-userLambda',
 'V-cheby-aggres-L1-trunc',
 'V-cheby-smoother',
 'V-cheby_poly-smoother',
 'agg_cheb4']

In [10]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_3d.problem, mesh_family["S5"]
)

In [11]:
results = []
for config in tqdm.tqdm(working_configs):
    cfg_json = configs[config]
    cfg = pyamgx.Config().create_from_dict(cfg_json)
    results.append(
        {
            "config": config,
            **test_config(
                cfg, double_precision_discrete_problem, rep_setup=1, rep_solve=3
            ),
        }
    )

df2 = pd.DataFrame(results)

 91%|█████████▏| 21/23 [00:58<00:08,  4.48s/it]!!! detected some memory leaks in the code: trying to free non-empty temporary device pool !!!
ptr:     0x7276043c9000 size: 4096
100%|██████████| 23/23 [01:10<00:00,  3.07s/it]


In [12]:
stats5 = df2.pivot_table(
    index="config", values=["setup_time", "solve_time", "residual"], aggfunc="min"
)
stats5

,residual,setup_time,solve_time
config,,,
AGGREGATION_GS,4.428876e-10,0.042540,1.338031
AGGREGATION_LOW_DEG_GS,4.428876e-10,0.038570,1.336580
AGGREGATION_THRUST_GS,5.712295e-10,0.037964,1.152502
AMG_AGGRREGATION_CG,3.606397e-10,0.015403,5.487268
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,2.166879e-10,0.050410,0.082629
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,3.652038e-10,0.049766,0.065653
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,3.104022e-10,0.051064,0.069127
AMG_CLASSICAL_L1_AGGRESSIVE_HMIS__MOD_PCG,3.449247e-10,3.238883,0.074500
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,3.435473e-10,0.081232,0.077249


In [13]:
fast_solvers = list(
    stats5[
        (stats5.solve_time < 2 * stats5.solve_time.min()) & (stats5.residual < 1e-06)
    ].index
)
fast_solvers

['AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC',
 'AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG',
 'AMG_CLASSICAL_L1_AGGRESSIVE_HMIS__MOD_PCG',
 'AMG_CLASSICAL_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_PMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_HMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG',
 'GMRES_AMG_D2__MOD_PCG']

In [14]:
double_precision_discrete_problem = discretize(
    continuous_coefficient_3d.problem, mesh_family["S7"]
)

In [15]:
# HMIS Thrust errors for large problems
fast_solvers = [solver for solver in fast_solvers if not "HMIS" in solver]
fast_solvers

['AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC',
 'AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG',
 'AMG_CLASSICAL_L1_TRUNC__MOD_PCG',
 'AMG_CLASSICAL_PMIS__MOD_PCG',
 'FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG',
 'GMRES_AMG_D2__MOD_PCG']

In [16]:
try:
    results = []
    for config in tqdm.tqdm(fast_solvers):
        cfg_json = configs[config]
        cfg = pyamgx.Config().create_from_dict(cfg_json)
        results.append(
            {
                "config": config,
                **test_config(
                    cfg, double_precision_discrete_problem, rep_setup=3, rep_solve=30
                ),
            }
        )

    df3 = pd.DataFrame(results)
except Exception as e:
    print("Caught runtime error:", e)

100%|██████████| 7/7 [14:28<00:00, 124.07s/it]


In [17]:
stats7 = df3.pivot_table(
    index="config", values=["setup_time", "solve_time", "residual"], aggfunc="min"
)
stats7

,residual,setup_time,solve_time
config,,,
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,4.799332e-11,2.375817,4.064625
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,4.175777e-11,2.372854,3.209093
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,3.612749e-11,2.452867,2.792076
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,3.948690e-11,4.327482,4.128526
AMG_CLASSICAL_PMIS__MOD_PCG,4.175777e-11,2.375387,3.210780
FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG,4.175777e-11,2.375021,3.211267
GMRES_AMG_D2__MOD_PCG,3.350606e-11,2.207326,5.271732


In [18]:
stats7.sort_values("solve_time")

,residual,setup_time,solve_time
config,,,
AMG_CLASSICAL_AGGRESSIVE_L1__MOD_PCG,3.612749e-11,2.452867,2.792076
AMG_CLASSICAL_AGGRESSIVE_L1_TRUNC__MOD_PCG,4.175777e-11,2.372854,3.209093
AMG_CLASSICAL_PMIS__MOD_PCG,4.175777e-11,2.375387,3.210780
FGMRES_CLASSICAL_AGGRESSIVE_PMIS__MOD_PCG,4.175777e-11,2.375021,3.211267
AMG_CLASSICAL_AGGRESSIVE_CHEB_L1_TRUNC,4.799332e-11,2.375817,4.064625
AMG_CLASSICAL_L1_TRUNC__MOD_PCG,3.948690e-11,4.327482,4.128526
GMRES_AMG_D2__MOD_PCG,3.350606e-11,2.207326,5.271732


In [19]:
stats7.to_csv("../results/amgx_selection_3d.csv")